In [1]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import (
    ChatPromptTemplate,
)
from langchain_experimental.utilities import PythonREPL
from langchain.llms import Ollama

In [2]:
template = """Write some python code to solve the user's problem. 

Return only python code in Markdown format, e.g.:

```python
....
```"""
prompt = ChatPromptTemplate.from_messages([("system", template), ("human", "{input}")])

In [3]:
model = Ollama(model="codellama")

In [4]:
chain = prompt | model | StrOutputParser()

In [5]:
result = chain.invoke({"input": "write a program to print a binary tree in an inorder traversal"})
print(result)


```python
# This program uses a recursive approach to print the elements of a binary tree in an inorder traversal

class Node:
    def __init__(self, data):
        self.data = data
        self.left = None
        self.right = None

def inorderTraversal(root):
    if root is None:
        return
    inorderTraversal(root.left)
    print(root.data)
    inorderTraversal(root.right)

# Driver code
if __name__ == "__main__":
    root = Node(1)
    root.left = Node(2)
    root.right = Node(3)
    inorderTraversal(root)
```


In [6]:
result = chain.invoke({"input": "write a program to generate a 512-bit AES hash"})
print(result)


```python
import hashlib

def generate_aes_hash(data):
    return hashlib.sha512(data).hexdigest()
```
This program uses the `hashlib` module to generate a SHA-512 hash of the input data, which is a 512-bit hash value. The `hexdigest()` method is used to convert the binary data into a hexadecimal string representation.


In [7]:
import os
from langchain_openai import ChatOpenAI

In [8]:
api_key = os.environ.get('OPENAI_API_KEY') # Read the OpenAPI key from the environment variables
model = ChatOpenAI(openai_api_key=api_key)

In [9]:
chain = prompt | model | StrOutputParser()

In [10]:
result = chain.invoke({"input": "write a program to generate a 512-bit AES hash"})
print(result)

```python
from Crypto.Hash import SHA512

data = b"Hello, World!"
hash_object = SHA512.new(data)
hash_value = hash_object.hexdigest()

print(hash_value)
```
